# Advanced Guide on Computing Electron Integrals

This tutorial is for demonstrating how to obtain the molecular orbital (MO) electron integrals as well as the molecular Hamiltonian.

Here, we adopt the physicists' convention for the electron integrals, so that they are related to the Hamiltonian by the equation:

\begin{equation}
H = E_{\text{nuc}} + \sum_{i, j = 1}^{N_{\text{spin orbitals}}} h_{ij} c_i^{\dagger} c_j + \frac{1}{2} \sum_{i, j, k, l = 1}^{N_{\text{spin orbitals}}} h_{ijkl} c_i^{\dagger} c_j^{\dagger} c_k c_l, \nonumber
\end{equation}

where 

- $E_{\text{nuc}}$ is the nuclear repulsion energy.
- $h_{ij}$ is the 1-electron MO integral (physicist's convention).
- $h_{ijkl}$ is the 2-electron MO integral (physicist's convention).
- $c_i^{\dagger}$, $c_i$ are the fermionic creation and annihilation operators on the i-th _spin_ orbtial.

## Prerequisite

QURI Parts modules used in this tutorial: `quri-parts-chem`, `quri-parts-pyscf`, `quri-parts-openfermion`. You can install them as follows:

In [1]:
#!pip install "quri_parts[chem]"
#!pip install "quri_parts[pyscf]"
#!pip install "quri_parts[openfermion]"

## Defining the Molecule

In quri-parts, one can create the molecule with the `PySCF` library and store it inside a `PySCFMolecularOrbitals`.

In [2]:
from pyscf import gto, scf
from quri_parts.pyscf.mol import PySCFMolecularOrbitals

h2o_atom_list = [['H', [0, 0, 0]], ['O', [2, 0, 1]], ['H',  [0, 0, 2]]]
h2o_mol = gto.Mole(atom=h2o_atom_list, verbose = 0)
h2o_mol.build()
h2o_mf = scf.RHF(h2o_mol)
h2o_mf.kernel()
h2o_mo = PySCFMolecularOrbitals(mol=h2o_mol, mo_coeff=h2o_mf.mo_coeff)

## Computing the electron integrals with `quri_parts`

### Computing the spatial electron integrals with `quri_parts`

In `quri_parts`, atomic orbital integrals are stored inside `AOeIntArraySet`, which contains:
    
- The nuclear repulsion energy of a molecule. 
- An `AO1eIntArray` object, which contains the array of the one-electron atomic orbital integral.
- An `AO2eIntArray` object, which contains the array of the two-electron atomic orbital integral.

One can construct the `AOeIntArraySet` using the `get_ao_eint_set` function. When the `store_array_on_memory` argument is set to True,it returns an `AOeIntArraySet` object, that stores the electron integrals on memory.

In [3]:
from quri_parts.pyscf.mol import get_ao_eint_set

ao_eint_set = get_ao_eint_set(h2o_mo, store_array_on_memory=True)

# the nuclear repulsion energy
nuc_energy = ao_eint_set.constant
# the ao one-electron integral: an AO1eIntArray object
ao_1e_int = ao_eint_set.ao_1e_int
# the ao two-electron integral: an AO2eIntArray object
ao_2e_int = ao_eint_set.ao_2e_int

One can access the explicit array of the electron atomic orbital integrals with the array attribute:


In [4]:
# ao_1e_int.array
# ao_2e_int.array

In practical quantum chemistry application, it is often the case that the electron molecular orbital (MO) integrals are more useful for construting the molecular Hamiltonian. 

The spatial mo electron integrals are stored inside `SpatialMOeIntSet`. It contains

- The nuclear repulsion energy of a molecule. 
- A `SpatialMO1eIntArray` object, which contains the array of the one-electron spatial molecular orbital integral.
- A `SpatialMO2eIntArray` object, which contains the array of the two-electron spatial molecular orbital integral.

<a id='array_based_mo_eint'></a>
These spatial electron integrals are computed via the `to_spatial_mo1int` and `to_spatial_mo2int` methods inside `AO1eInt` and `AO2eInt`, respectively.

In [5]:
from quri_parts.chem.mol import SpatialMOeIntSet

# Construct a SpatialMO1eIntArray instance
spatial_mo_1e_int = ao_1e_int.to_spatial_mo1int(h2o_mo.mo_coeff)

# Construct a SpatialMO2eIntArray instance
spatial_mo_2e_int = ao_2e_int.to_spatial_mo2int(h2o_mo.mo_coeff)

# Store them inside a SpatialMOeIntSet object
spatial_mo_eint_set = SpatialMOeIntSet(
    const=nuc_energy,
    mo_1e_int=spatial_mo_1e_int,
    mo_2e_int=spatial_mo_2e_int
)


# The explicit arrays can be obtained by accessing the array attribute.

# spatial_mo_1e_int.array
# spatial_mo_2e_int.array

If one wishes to compute the spin space mo integrals, then one can use `to_mo1int` and `to_mo2int` methods and store them inside the `SpinMOeIntSet` class.
<a id='array_based_spin_mo_eint'></a>

In [6]:
from quri_parts.chem.mol import SpinMOeIntSet

# Construct a SpinMO1eIntArray instance
spin_mo_1e_int = ao_1e_int.to_mo1int(h2o_mo.mo_coeff)

# Construct a SpinMO2eIntArray instance
spin_mo_2e_int = ao_2e_int.to_mo2int(h2o_mo.mo_coeff)

# Store them inside a SpinMOeIntSet object
spin_mo_eint_set = SpinMOeIntSet(
    const=nuc_energy,
    mo_1e_int=spin_mo_1e_int,
    mo_2e_int=spin_mo_2e_int
)

# The explicit arrays can be obtained by accessing the array attribute.

# spin_mo_1e_int.array
# spin_mo_2e_int.array

These can also be obtained in a simpler way with the `to_full_space_spatial_mo_int` and `to_full_space_mo_int` methods in `ao_eint_set`

In [7]:
# Computes the full space spatial mo electron integrals
spatial_mo_e_int_set = ao_eint_set.to_full_space_spatial_mo_int(h2o_mo)

# Computes the full space spin mo electron integrals
spin_mo_e_int_set = ao_eint_set.to_full_space_mo_int(h2o_mo)

### Computing the active space electron integrals

<a id='active_space'></a>

In quri-parts, we can specify the active space with the `ActiveSpace` class, and the active space molecular orbitals can be constructed from the `ActiveSpaceMolecularOrbital` class.

In [8]:
from quri_parts.chem.mol import ActiveSpace, ActiveSpaceMolecularOrbitals

n_active_ele = 6
n_active_orb = 4

active_space = ActiveSpace(n_active_ele=n_active_ele, n_active_orb=n_active_orb)
active_space_mo = ActiveSpaceMolecularOrbitals(mo=h2o_mo, active_space=active_space)

Detailed information of the active space can be obtained by printing out the active_space_mo. 

Note that `quri-parts` would perform consistency checks on the active space automatically to make sure the active space configuration is correct.

In [9]:
print(active_space_mo)

n_electron: 10
n_active_ele: 6
n_core_ele: 4
n_ele_alpha: 3
n_ele_beta: 3
n_spatial_orb: 7
n_active_orb: 4
n_core_orb: 2
n_vir_orb: 1


To obtain the active space spin or spatial electron integrals, we can use the `get_active_space_spin_integrals_from_ao_eint` and `get_active_space_spatial_integrals_from_ao_eint` functions.

In [10]:
from quri_parts.chem.mol import (
    get_active_space_spin_integrals_from_ao_eint,
    get_active_space_spatial_integrals_from_ao_eint
)

# Computes the active space spin mo electron integrals
active_space_spin_integrals = get_active_space_spin_integrals_from_ao_eint(
    active_space_mo=active_space_mo,
    electron_ao_ints=ao_eint_set
)

# Computes the active space spatial mo electron integrals
active_space_spatial_integrals = get_active_space_spatial_integrals_from_ao_eint(
    active_space_mo=active_space_mo,
    electron_ao_ints=ao_eint_set,
)

Note that it is faster if one chooses to store the spatial mo electron integral on memory, and calls the `get_active_space_integrals_from_mo` as the mo integrals do not need to be computed again.

In [11]:
from quri_parts.chem.mol import (
    get_active_space_spatial_integrals_from_mo_eint,
    get_active_space_spin_integrals_from_mo_eint
)

# Computes the active space spin mo electron integrals
active_space_spin_integrals = get_active_space_spin_integrals_from_mo_eint(
    active_space_mo=active_space_mo,
    electron_mo_ints=spatial_mo_eint_set  # MOeIntSet is passed in here instead of AOeIntSet
)

# Computes the active space spatial mo electron integrals
active_space_spatial_integrals = get_active_space_spatial_integrals_from_mo_eint(
    active_space_mo=active_space_mo,
    electron_mo_ints=spatial_mo_eint_set,  # MOeIntSet is passed in here instead of AOeIntSet
)

The `AOeIntSet` also provides convenient methods to obtain the active space spin/spatial mo electron integrals right after
constructing `ao_eint_set`.

In [12]:
# Convenient method of computing the active space spin mo electron integrals
active_space_spin_integrals = ao_eint_set.to_active_space_mo_int(active_space_mo=active_space_mo)

# Convenient method of computing the active space spatial mo electron integrals
active_space_spatial_integrals = ao_eint_set.to_active_space_spatial_mo_int(active_space_mo=active_space_mo)

## Computing the electron integrals with `quri_parts` in a memory efficient manner

When the molecule gets larger, it is unrealistic to store the full space mo electron integrals on memory, most especially when we just want to get the hamiltonian of a small active space from a big molecule.

In quri-parts, we provide a memory efficient way of computing the active space integrals while bypassing the full space electron integrals.

### Computing the spatial electron integrals with `quri_parts`

The idea is to store only the pyscf Mole object inside the `PySCFAO1eInt`, `PySCFAO2eInt` classes, which are contained by a `PySCFAOeIntSet` object.

We may obtain the `PySCFAOeIntSet` with the `get_ao_eint_set` function

In [13]:
# This constructs a PySCFAOeIntSet object, which only holds pyscf mol object on memory
pyscf_ao_eint_set = get_ao_eint_set(h2o_mo)

Importantly, the explicit eletron integrals are only computed on demand whenever the `array` property is accessed.

In [14]:
# pyscf_ao_eint_set.ao_1e_int.array
# pyscf_ao_eint_set.ao_2e_int.array

The corresponding spatial mo integrals can be computed by the methods `to_spatial_mo1int` and `to_spatial_mo2int` methods, which returns the `PySCFSpatialMO1eInt` and `PySCFSpatialMO2eInt` objects that contains a pyscf mol object and the mo coefficients.

In [15]:
# Returns a PySCFSpatialMO1eInt object, which only holds the molecule and the mo coefficients on memory
pyscf_spatial_mo_1e_int = pyscf_ao_eint_set.ao_1e_int.to_spatial_mo1int(h2o_mo.mo_coeff)

# Returns a PySCFSpatialMO2eInt object, which only holds the molecule and the mo coefficients on memory
pyscf_spatial_mo_2e_int = pyscf_ao_eint_set.ao_2e_int.to_spatial_mo2int(h2o_mo.mo_coeff)

The explicit electron integrals are only computed on demand when the `array` property is accessed

In [16]:
# pyscf_spatial_mo_1e_int.array
# pyscf_spatial_mo_2e_int.array

We may show that the results are consistent with those from the `ao_1e_int.to_sptatial_mo1int` and `ao_2e_int.to_sptatial_mo2int` methods we showed [previously](#array_based_mo_eint), where the computations there are based on matrix multiplication.

In [17]:
from numpy import allclose
assert allclose(pyscf_spatial_mo_1e_int.array, spatial_mo_1e_int.array)
assert allclose(pyscf_spatial_mo_2e_int.array, spatial_mo_2e_int.array)

The corresponding spin mo integrals can be computed by the methods `to_mo1int` and `to_mo2int` methods,

which returns the `SpinMO1eInt` and `SpinMO2eInt` objects that only holds the PySCF Mole object and the mo coefficient array on memory.

In [18]:
# Returns a SpinMO1eInt object that holds the full space electron integral arrays
pyscf_spin_mo_1e_int = pyscf_ao_eint_set.ao_1e_int.to_mo1int(h2o_mo.mo_coeff)

# Returns a SpinMO2eInt object that holds the full space electron integral arrays
pyscf_spin_mo_2e_int = pyscf_ao_eint_set.ao_2e_int.to_mo2int(h2o_mo.mo_coeff)

The computation of the actual electron integral are only computed when the `array` property is accessed.

In [19]:
# pyscf_spin_mo_1e_int.array  # electron integral computation happens here when .array is accessed
# pyscf_spin_mo_2e_int.array  # electron integral computation happens here when .array is accessed

We may show that the results are consistent with those from the `ao_1e_int.to_mo1int` and `ao_2e_int.to_mo2int` methods we showed [previously](#array_based_spin_mo_eint),

In [20]:
from numpy import allclose
assert allclose(pyscf_spin_mo_1e_int.array, spin_mo_1e_int.array)
assert allclose(pyscf_spin_mo_2e_int.array, spin_mo_2e_int.array)

### Getting the active space spin electron integrals

For constructing the active space hamiltonian, we may use the `get_active_space_integrals` function in the `quri-parts.pyscf` package. 

It only takes in a `PySCFAOeIntSet` object and an `ActiveSpaceMolecularOribtal` object, which we constrcuted [previously](#active_space). 

Both of them do not take up mach space on memory.

In [21]:
import quri_parts.pyscf.mol as pyscf_mol

# Returns the active space spin mo electron integrals
pyscf_active_space_spin_integrals = pyscf_mol.get_active_space_spin_integrals(
    active_space_mo=active_space_mo, 
    electron_ints=pyscf_ao_eint_set  # takes in a PySCFAOeIntSet objet
)

# Returns the active space spatial mo electron integrals
pyscf_active_space_spatial_integrals = pyscf_mol.get_active_space_spatial_integrals(
    active_space_mo=active_space_mo,
    electron_ints=pyscf_ao_eint_set,  # takes in a PySCFAOeIntSet objet
)

We may also show that the result is consistent with that from the `get_active_space_integrals` function

In [22]:
assert allclose(pyscf_active_space_spin_integrals.const, active_space_spin_integrals.const)
assert allclose(pyscf_active_space_spin_integrals.mo_1e_int.array, active_space_spin_integrals.mo_1e_int.array)
assert allclose(pyscf_active_space_spin_integrals.mo_1e_int.array, active_space_spin_integrals.mo_1e_int.array)

`quri-parts` also provides convenient methods of getting the spin/spatial active space integral right after the construction of a `PySCFAOeIntSet` object:

In [23]:
# Convenient method of computing the active space spin mo electron integrals
pyscf_active_space_spin_integrals = pyscf_ao_eint_set.to_active_space_mo_int(active_space_mo)

# Convenient method of computing the active space spatial mo electron integrals
pyscf_active_space_spatial_integrals = pyscf_ao_eint_set.to_active_space_spatial_mo_int(active_space_mo)

## Computing the Molecular Hamiltonian

In `quri-parts-pyscf`, we provide the `get_spin_mo_integrals_from_mole` function for simply obtaining the spin MO electron integral (a `SpinMOeIntSet` object) along with the corresponding `MolecularOrbitals` object after constructing the pyscf molecule.

In [24]:
h2o_atom_list = [['H', [0, 0, 0]], ['O', [2, 0, 1]], ['H',  [0, 0, 2]]]
h2o_mol = gto.M(atom=h2o_atom_list, verbose = 0)
h2o_mf = scf.RHF(h2o_mol)
h2o_mf.kernel()

# The full space molecular orbitals and full space mo electron integrals
from quri_parts.pyscf.mol import get_spin_mo_integrals_from_mole
full_space_mo, full_space_spin_mo_eint_set = get_spin_mo_integrals_from_mole(h2o_mol, h2o_mf.mo_coeff)

The obtained mo electron integral coefficients can be used to construct the molecular hamiltonian.

For constrcuting the fermionic hamiltonian, we can use the `get_fermionic_hamiltonian` in `quri-parts-openfermion` that converts the MO electron integral into the fermionic operator represented by `Openfermion`'s `InteractionOperator`.

In [25]:
from quri_parts.openfermion.mol import get_fermionic_hamiltonian

fermionic_hamiltonian = get_fermionic_hamiltonian(full_space_spin_mo_eint_set)

To convert the fermionic hamiltonian into a `quri-parts` `Operator` with a specified fermion-qubit mapping method, we provide `convert_fermionic_hamiltonian_to_qubit_hamiltonian` in `quri-parts-openfermion`. The operator mapper and state mapper are also returned along with the qubit_hamiltonian.

In [26]:
from quri_parts.openfermion.mol import convert_fermionic_hamiltonian_to_qubit_hamiltonian
from quri_parts.openfermion.transforms import jordan_wigner
(
    qubit_hamiltonian,
    operator_mapper,
    state_mapper
) = convert_fermionic_hamiltonian_to_qubit_hamiltonian(
    fermionic_hamiltonian=fermionic_hamiltonian,
    mo=full_space_mo,
    fermion_qubit_mapping=jordan_wigner  # default to jordan_wigner
)

We also provide the `get_qubit_mapped_hamiltonian_operator` function in `quri-parts-openfermion` for obtaining the qubit Hamiltonian right after the MO electron integral is obtained.

In [27]:
from quri_parts.openfermion.mol import get_qubit_mapped_hamiltonian_operator
(
    qubit_hamiltonian,
    operator_mapper,
    state_mapper
) = get_qubit_mapped_hamiltonian_operator(
    mo=full_space_mo,
    spin_mo_eint_set=full_space_spin_mo_eint_set,
    fermion_qubit_mapping=jordan_wigner  # default to jordan_wigner
)

The active space hamiltonian can also be obtained as in the full space hamiltonian above.

In [28]:
from quri_parts.chem.mol import cas

# Obtaining the active space molecular orbitals
active_space = cas(n_active_ele=2, n_active_orb=2)
(
    active_space_mo,
    active_space_spin_mo_eint_set
) = get_spin_mo_integrals_from_mole(h2o_mol, h2o_mf.mo_coeff, active_space)

# Compute the active space fermionic hamiltonian and convert it to qubit hamiltonian
active_space_fermionic_hamiltonian = get_fermionic_hamiltonian(active_space_spin_mo_eint_set)
(
    active_space_qubit_hamiltonian,
    active_space_operator_mapper,
    active_space_state_mapper
) = convert_fermionic_hamiltonian_to_qubit_hamiltonian(active_space_fermionic_hamiltonian, active_space_mo)

Or equivalently, the qubit hamiltonian can be obtained right after the active space MO electron integral and `ActiveSpaceMolecularOrbitals` object are obtained.

In [29]:
(
    active_space_qubit_hamiltonian,
    operator_mapper,
    state_mapper
) = get_qubit_mapped_hamiltonian_operator(
    active_space_mo,
    active_space_spin_mo_eint_set
)

for term, coeff in active_space_qubit_hamiltonian.items():
    print(coeff, term)

(-74.19897804639758+0j) I
(0.010450191939802053+0j) Z0
(0.010450191939802053+0j) Z1
(0.05612455054746367+0j) Z2
(0.05612455054746367+0j) Z3
(0.11022849870581763+0j) Z0 Z1
(0.07024413153657523+0j) Z0 Z2
(0.10866454047937474+0j) Z0 Z3
(0.10866454047937474+0j) Z1 Z2
(0.07024413153657523+0j) Z1 Z3
(0.1106228519330524+0j) Z2 Z3
-0.0384204089427995 X0 X1 Y2 Y3
0.0384204089427995 X0 Y1 Y2 X3
0.0384204089427995 Y0 X1 X2 Y3
-0.0384204089427995 Y0 Y1 X2 X3


## Summary

Finally we summarize the necessary procedure for constructing the electron integrals for a molecule

### Compute with storing the electron integral arrays on memory

In [30]:
# Step 1: Construct a PySCF Mole object
from pyscf import gto, scf
from quri_parts.pyscf.mol import PySCFMolecularOrbitals

h2o_atom_list = [['H', [0, 0, 0]], ['O', [2, 0, 1]], ['H',  [0, 0, 2]]]
h2o_mol = gto.Mole(atom=h2o_atom_list, verbose = 0)
h2o_mol.build()
h2o_mf = scf.RHF(h2o_mol)
h2o_mf.kernel()
h2o_mo = PySCFMolecularOrbitals(mol=h2o_mol, mo_coeff=h2o_mf.mo_coeff)

In [31]:
# Step 2: Compute the ao electron integrals
from quri_parts.pyscf.mol import get_ao_eint_set

ao_eint_set = get_ao_eint_set(h2o_mo, store_array_on_memory=True)

# If you only wish to get the full space electron integrals, you can run:
spin_mo_eint_set = ao_eint_set.to_full_space_mo_int(h2o_mo)
# and ignore Step 3 and Step 4.

In [32]:
# Step 3: Construct the desired active space
from quri_parts.chem.mol import ActiveSpaceMolecularOrbitals, cas

active_space_mo = ActiveSpaceMolecularOrbitals(mo=h2o_mo, active_space=cas(n_active_ele=6, n_active_orb=4))

In [33]:
# Step 4: Compute the active space spin mo integrals
from quri_parts.chem.mol import get_active_space_spin_integrals_from_ao_eint, get_active_space_spatial_integrals_from_ao_eint

active_space_spatial_mo_eint_set = get_active_space_spatial_integrals_from_ao_eint(active_space_mo=active_space_mo, electron_ao_ints=ao_eint_set)
active_space_spin_mo_eint_set = get_active_space_spin_integrals_from_ao_eint(active_space_mo=active_space_mo, electron_ao_ints=ao_eint_set)

# Or we can also do:
active_space_spatial_mo_eint_set = ao_eint_set.to_active_space_spatial_mo_int(active_space_mo)
active_space_spin_mo_eint_set = ao_eint_set.to_active_space_mo_int(active_space_mo)

### Compute without storing the electron integral arrays on memory

In [34]:
# Step 1: Construct a PySCF Mole object
from pyscf import gto, scf
from quri_parts.pyscf.mol import PySCFMolecularOrbitals

h2o_atom_list = [['H', [0, 0, 0]], ['O', [2, 0, 1]], ['H',  [0, 0, 2]]]
h2o_mol = gto.Mole(atom=h2o_atom_list, verbose = 0)
h2o_mol.build()
h2o_mf = scf.RHF(h2o_mol)
h2o_mf.kernel()
h2o_mo = PySCFMolecularOrbitals(mol=h2o_mol, mo_coeff=h2o_mf.mo_coeff)

In [35]:
# Step 2: Compute the ao electron integrals without storing the electron integral arrays on memory
from quri_parts.pyscf.mol import get_ao_eint_set

pyscf_ao_eint_set = get_ao_eint_set(h2o_mo)

# If you wish to get the full space electron integrals, you can run:
pyscf_spin_mo_eint_set = pyscf_ao_eint_set.to_full_space_mo_int(h2o_mo)
# and ignore Step 3 and Step 4.

# Note that pyscf_spin_mo_eint_set only stores PySCF Mole object and the mo coefficients on memory.
# The explicit array can be obtained by accessing the `array` property
full_space_mo_1e_int_array = pyscf_spin_mo_eint_set.mo_1e_int.array  # electron integral computations happen here
full_space_mo_2e_int_array = pyscf_spin_mo_eint_set.mo_2e_int.array  # electron integral computations happen here

In [36]:
# Step 3: Construct the desired active space
from quri_parts.chem.mol import ActiveSpaceMolecularOrbitals, cas

active_space_mo = ActiveSpaceMolecularOrbitals(mo=h2o_mo, active_space=cas(n_active_ele=6, n_active_orb=4))

In [37]:
# Step 4: Compute the active space spin mo integrals
from quri_parts.pyscf.mol import get_active_space_spin_integrals, get_active_space_spatial_integrals

pyscf_active_space_spatial_mo_eint_set = get_active_space_spatial_integrals(
    active_space_mo,
    pyscf_ao_eint_set
)
pyscf_active_space_spin_mo_eint_set = get_active_space_spin_integrals(
    active_space_mo,
    pyscf_ao_eint_set
)

# Or equivalently, we can also do:
pyscf_active_space_spatial_mo_eint_set = pyscf_ao_eint_set.to_active_space_spatial_mo_int(
    active_space_mo
)
pyscf_active_space_spin_mo_eint_set = pyscf_ao_eint_set.to_active_space_mo_int(
    active_space_mo
)

### Obtaining the hamiltonian

In [38]:
# Step 1: Construct a PySCF Mole object

from pyscf import gto, scf
from quri_parts.pyscf.mol import PySCFMolecularOrbitals

h2o_atom_list = [['H', [0, 0, 0]], ['O', [2, 0, 1]], ['H',  [0, 0, 2]]]
h2o_mol = gto.M(atom=h2o_atom_list, verbose = 0)
h2o_mf = scf.RHF(h2o_mol)
h2o_mf.kernel()

-74.24459532257532

In [48]:
from quri_parts.pyscf.mol import get_spin_mo_integrals_from_mole

# Step 2 (for full space hamiltonian):
# Construct the MolecularObitals object and the MO electron integrals
full_space_mo, full_space_spin_mo_eint_set = get_spin_mo_integrals_from_mole(
    mole=h2o_mol,
    mo_coeff=h2o_mf.mo_coeff
)

# Step 3 (for full space hamiltonian): 
# Obtain the qubit hamiltonian
qubit_hamiltonian = get_qubit_mapped_hamiltonian_operator(
    mo=full_space_mo,
    spin_mo_eint_set=full_space_spin_mo_eint_set,
    fermion_qubit_mapping=jordan_wigner  # default to jordan_wigner
)

In [46]:
from quri_parts.pyscf.mol import get_spin_mo_integrals_from_mole


# Step 2 (for active space hamiltonian):
# Construct the MolecularObitals object and the MO electron integrals
active_space_mo, active_space_spin_mo_eint_set = get_spin_mo_integrals_from_mole(
    mole=h2o_mol,
    mo_coeff=h2o_mf.mo_coeff,
    active_space=cas(2, 2)
)

# Step 3 (for active space hamiltonian): 
# Obtain the qubit hamiltonian
active_space_qubit_hamiltonian = get_qubit_mapped_hamiltonian_operator(
    mo=active_space_mo,
    spin_mo_eint_set=active_space_spin_mo_eint_set,
    fermion_qubit_mapping=jordan_wigner  # default to jordan_wigner
)

### Run checks (Should be remove when this tutorial is finalized)

In [41]:
from numpy import allclose

In [42]:
assert allclose(pyscf_ao_eint_set.to_full_space_mo_int(mo=h2o_mo).const, ao_eint_set.to_full_space_mo_int(mo=h2o_mo).const)
assert allclose(pyscf_ao_eint_set.to_full_space_mo_int(mo=h2o_mo).mo_1e_int.array, ao_eint_set.to_full_space_mo_int(mo=h2o_mo).mo_1e_int.array)
assert allclose(pyscf_ao_eint_set.to_full_space_mo_int(mo=h2o_mo).mo_2e_int.array, ao_eint_set.to_full_space_mo_int(mo=h2o_mo).mo_2e_int.array)

In [43]:
assert allclose(pyscf_ao_eint_set.to_full_space_spatial_mo_int(mo=h2o_mo).const, ao_eint_set.to_full_space_spatial_mo_int(mo=h2o_mo).const)
assert allclose(pyscf_ao_eint_set.to_full_space_spatial_mo_int(mo=h2o_mo).mo_1e_int.array, ao_eint_set.to_full_space_spatial_mo_int(mo=h2o_mo).mo_1e_int.array)
assert allclose(pyscf_ao_eint_set.to_full_space_spatial_mo_int(mo=h2o_mo).mo_2e_int.array, ao_eint_set.to_full_space_spatial_mo_int(mo=h2o_mo).mo_2e_int.array)

In [ ]:
assert allclose(active_space_spin_mo_eint_set.const, pyscf_active_space_spin_mo_eint_set.const)
assert allclose(active_space_spin_mo_eint_set.mo_1e_int.array, pyscf_active_space_spin_mo_eint_set.mo_1e_int.array)
assert allclose(active_space_spin_mo_eint_set.mo_2e_int.array, pyscf_active_space_spin_mo_eint_set.mo_2e_int.array)